# Retreiving and Preparing Text for Machines

In [5]:
#Special module written for this class
#This provides access to data and to helper functions from previous weeks
import lucem_illud #pip install git+git://github.com/Computational-Content-Analysis-2018/lucem_illud.git

#All these packages need to be installed from pip
import requests #for http requests
import bs4 #called `beautifulsoup4`, an html parser
import pandas as pd#gives us DataFrames
import docx #reading MS doc files, install as `python-docx`

#Stuff for pdfs
#Install as `pdfminer2`
import pdfminer.pdfinterp
import pdfminer.converter
import pdfminer.layout
import pdfminer.pdfpage

#These come with Python
import re #for regexs
import urllib 
import io #for making http requests look like files
import json #For Tumblr API responses
import os.path #For checking if files exist
import os #For making directories

from IPython.display import display
#from bs4 import BeautifulSoup, NavigableString

### Scraping Text (from saved .html files)

In [62]:
countryDict = {
'US': [
"us01.html",
"us02.html",
"us03.html",
"us04.html",
"us05.html",
"us06.html",
"us07.html",
"us08.html",
"us09.html",
"us10.html"
]}              

In [63]:
def loopThruArticles(countryDict):

    parsDict = {'country': [], 'source': [], 'title': [], 'text': [], 'date': [], 'author': [], 'section': []}

    for key, val in countryDict.items():
        for html in val:
            parsDict['country'].append(key)
            parseArticle(html, parsDict)
    
    return parsDict
    

In [160]:
def parseArticle(html, parsDict):
    soup = bs4.BeautifulSoup(open(html, 'rt', encoding='UTF8'),'lxml')

    # title
    title0 = soup.find_all('div', {'class' : 'title'})
    title1 = title0[0].text.replace('Hide Details', '') #cleaning
    title = re.sub(r'\<.*\>', '', title1)
    parsDict['title'].append(title)

    # text
    body = soup.find_all('div', {'class' : 'body'})
    text0 = "".join([t for t in body[0].contents if type(t)==bs4.element.NavigableString]) #get text only from content, not children
    text1 = re.sub(r"Source- .+","", text0) #cleaning
    text2 = re.sub(r"© .+","", text1) #cleaning
    text3 = re.sub(r"(STORY CAN END HERE)","", text2) #cleaning 
    text4 = re.sub(r"(EDITORS: STORY CAN END HERE)","", text3) #cleaning
    text5 = re.sub(r"(EDITORS: )","", text4) #cleaning
    text = re.sub(r"^([A-Z]+)", "", text5) #take out first word if all caps (it is a location)
    parsDict['text'].append(text)
    
#     with open("%s.txt" % html[:-5], "w", encoding='UTF-8') as text_file:
#         text_file.write(text)

    # source & date
    source = soup.find_all('div', {'class' : 'source'})
    date = re.findall(r'(?:January|February|March|April|May|June|July|August|September|October|November|December)\s\d{1,2},\s\d{4}', source[0].text)
    parsDict['date'].append(date[0])
    pat2 = re.compile(r"(.*?)-\s", re.M)
    newssource = pat2.findall(source[0].text)
    parsDict['source'].append(newssource[0])

    # author  
    author0 = soup.find_all('span', {'class' : 'val'})
    author = re.sub(r'\<.*\>', '', author0[0].text) #Take out <>
    parsDict['author'].append(author)

    # section
    section0 = soup.find_all('span', {'class' : 'lbl'}, string="Section: ")
    section = section0[0].next_element.next_element.next_element.text
    #section = re.sub(r'\<.*\>', '', section0.text) #Take out <>
    parsDict['section'].append(section)
    

In [161]:
df = pd.DataFrame(loopThruArticles(countryDict))

In [162]:
df

,author,country,date,section,source,text,title
0,"Sheryl DeVore, News-Sun",US,"November 16, 2017",News,Libertyville Review (IL),ibertyville High School senior Leah Hartung le...,'No argument' climate\nchange man-made - \nFor...
1,"Sheryl DeVore, News-Sun",US,"November 16, 2017",News,Mundelein Review (IL),ibertyville High School senior Leah Hartung le...,'No argument' climate\nchange man-made - \nFor...
2,"Kirt Manion, kmanion@ncnewspress.com",US,"November 10, 2017",News,Nebraska City News-Press (NE),"hen people discuss the topic of \n, most of th...",UNL scientist discusses state impact of climat...
3,"Sheryl DeVore, News-Sun",US,"November 10, 2017",News,"Lake County News-Sun (Waukegan, IL)",ibertyville High School senior Leah Hartung le...,'No argument' climate\nchange man-made - \nFor...
4,"Stuart Leavenworth, Staff Writer",US,"November 5, 2017",News,Belleville News-Democrat (IL) (Published as Be...,The Trump administration released a sweeping ...,Climate\nchange sucks moisture from West
5,Stuart Leavenworth \nMcClatchy Washington Bureau,US,"November 4, 2017",A,"Skagit Valley Herald (Mount Vernon, WA)",— The Trump administration released a sweepin...,Fed study: Climate\nchange sucks moisture from...
6,LISA FRIEDMAN New York Times Service,US,"November 4, 2017",Front Page,"Daily Gazette, The (Schenectady, NY)",-- Directly contradicting much of the Trump ...,Climate report at odds with Trump
7,LISA FRIEDMAN,US,"November 4, 2017",News,"Tri-City Herald (Kennewick, WA)",Directly contradicting much of the Trump admi...,US report contradicts Trump on cause of climat...
8,CHANK KIEU theconversation.com,US,"September 20, 2017",News,"Times and Democrat, The (Orangeburg, SC)","urricane Harvey, with its historical amount of...",Do hurricanes feel the effects of climate\ncha...
9,Chanh Kieu \nIndiana University,US,"September 13, 2017",Science & Technology,Alva Review-Courier (OK),o hurricanes feel the effects of \n?\n(The Con...,Do hurricanes feel the effects of climate\ncha...
